# Rename and Orga files to apply preprocess pipeline

Needed per subject:

- *hardi_2mm_2b_as.rec & hardi_2mm_2b_ps.rec --> parec2nii (will give necessary bval/bves files and all 33 directions in one files)
- *hardi_2mm_2b_as_bvalue1_diffori33.nii & *hardi_2mm_2b_ps_bvalue1_diffori33.nii as b0 images necessary for topup (fieldmap correction)
- acqparams.txt = describing the phase encoding direction and echo time for TOPUP | checked the scan sequence, should be (according to ChatGPT given the info of the file) 
0  1  0  0.0215
0 -1  0  0.0215

### Infos:
https://fsl.fmrib.ox.ac.uk/fslcourse/2019_Beijing/lectures/FDT/fdt1.html


## Succesfull FA image generation:

1.	Transform the .par files to nifti while also creating the necessary .bval & .bvec files (parrec2nii --bvs *_ps.par)
2.	Merge all into on large 4D file (66 in dim4):  fslmerge -t dwi_all.nii.gz *_as.nii *_ps.nii
    a.	Also merge bvec &bval files: paste -d' ' dwi_as.bval dwi_ps.bval > dwi_all.bval (same for .bvecs)
3.	Merge b0 files: fslmerge -t b0_all sn_*_as_bvalue1_diffori33.nii sn_*_ps_bvalue1_diffori33.nii
4.	Get distortion corrections: topup --imain=b0_all --datain=acqparams.txt --config=b02b0.cnf --subsamp=1 --out=topup_results --iout=corrected_b0
5.	Apply topup results: applytopup --imain=dwi_all --topup=topup_results --datain=acqparams.txt --inindex=1 --method=jac --out=dwi_topup_corrected
6.	Get brain mask from mean dti image (the version where I generate the brain mask from corrected_b0 seems to misalign) :
    a.	fslmaths dwi_topup_corrected.nii.gz -Tmean dwi_topup_corrected_mean
    b.	bet dwi_topup_corrected_mean.nii.gz mask_dwi_topup_corrected_mean -m -f 0.2
7.	Fit diffusion tensor model at each voxel: dtifit --data=dwi_topup_corrected --mask=mask_dwi_topup_corrected_mean_mask --bvecs=dwi_all.bvecs --bvals=dwi_all.bvals --out=dtifit _meanmask_results


--> meeting with Zoltan & Tim
* run topup directly on all DTI ims?! empirical question, look into FSL forum on latest opinions on this
* play with BET params, visually check (every sub!?)

### 2ndl Level
* Zoltan does not like TBSS, rather apply some warp function, bring to standard space, test ROIs

### DTI images in BIDS format:

https://bids.neuroimaging.io/getting_started/folders_and_files/files.html#mri


sub-<label>/
    [ses-<label>/]
        dwi/
            sub-<label>[_ses-<label>][_acq-<label>][_rec-<label>][_dir-<label>][_run-<index>][_part-<mag|phase|real|imag>][_chunk-<index>]_dwi.bval
            sub-<label>[_ses-<label>][_acq-<label>][_rec-<label>][_dir-<label>][_run-<index>][_part-<mag|phase|real|imag>][_chunk-<index>]_dwi.bvec
            sub-<label>[_ses-<label>][_acq-<label>][_rec-<label>][_dir-<label>][_run-<index>][_part-<mag|phase|real|imag>][_chunk-<index>]_dwi.json
            sub-<label>[_ses-<label>][_acq-<label>][_rec-<label>][_dir-<label>][_run-<index>][_part-<mag|phase|real|imag>][_chunk-<index>]_dwi.nii[.gz]

parrec2nii --bvs *_ps.par

* Filename entities or directories between square brackets (for example, [_ses-<label>]) are OPTIONAL.

--> relevant: `dir` - Phase-Encoding Direction : The dir-<label> entity can be set to an arbitrary (!!) legitimate label (for example, dir-LR or dir-AP) to distinguish different phase-encoding directions. -- Therefore, if the dir-<label> entity is present in a filename, "PhaseEncodingDirection" MUST be defined in the associated metadata

## 1. Prep files

run the following for all subs

for sub in $(seq -w 1 66); do mkdir /Volumes/mrenkeED/data/ds-dnumrisk/sub-${sub}/ses-1/dwi ; done

for dir in as ps; do
  for sub in $(seq -w 1 66); do
    parrec2nii \
      /Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/mri/SNS_MRI_DNUMR_S000${sub}_01/*_${dir}.par \
      -o /Volumes/mrenkeED/data/ds-dnumrisk/sub-${sub}/ses-1/dwi \
      --bvs
  done
done

{$(seq -w 1 66) adds the 0 for 1-9!}

- Problems: sub-39; sub-64 (weird names, no ps/as specification --> assume that ps was done first (smaller acq number!))

In [ ]:
# Copy from sourcedata into sub-00/ses-1/anat and rename 

import glob
import re
import os
import os.path as op

import shutil

bids_folder = '/Volumes/mrenkeED/data/ds-dnumrisk'

for sub in range(1,67):
    sub = str(sub).zfill(2)
    pati = op.join(bids_folder,f'sub-{sub}','ses-1','dwi')
    files = os.listdir(pati)

    for file in files:
        ending = file.split('_')[-1] 
        dir = ending.split('.')[0] # take the last part of the filename (containing dir & filetye)
        f_type = ending.split('.')[1] # take the last part of the filename (containing dir & filetye)
        new_fn =  f'sub-{sub}_dir-{dir}_dwi.{f_type}'
        print(new_fn)

        os.rename(op.join(pati,file), op.join(pati,new_fn))



## 2. Merge all into on large 4D file (66 in dim4): 

for sub in $(seq -w 1 66); 

do mkdir /Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-${sub}

do fslmerge -t  /Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-${sub}/sub-${sub}_dir-all_dwi.nii /Volumes/mrenkeED/data/ds-dnumrisk/sub-${sub}/ses-1/dwi/sub-${sub}_dir-as_dwi.nii /Volumes/mrenkeED/data/ds-dnumrisk/sub-${sub}/ses-1/dwi/sub-${sub}_dir-ps_dwi.nii;  

done

for spec in bvals bvecs; do
  for sub in $(seq -w 1 66); do
    paste -d' ' \
    /Volumes/mrenkeED/data/ds-dnumrisk/sub-${sub}/ses-1/dwi/sub-${sub}_dir-as_dwi.${spec} \
    /Volumes/mrenkeED/data/ds-dnumrisk/sub-${sub}/ses-1/dwi/sub-${sub}_dir-ps_dwi.${spec} \
    > \
    /Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-${sub}/sub-${sub}_dir-all_dwi.${spec}
  done
done

--> this part is probably unnecessary, as I need these files only once cause they are the same for all subs

## 3.	Merge b0 files: 
fslmerge -t b0_all sn_*_as_bvalue1_diffori33.nii sn_*_ps_bvalue1_diffori33.nii

fslmerge -t /Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-${sub}/sub-${sub}_dir-all-b0_dwi \
/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/mri/SNS_MRI_DNUMR_S000${sub}_01/*_as_bvalue1_diffori33.nii \
/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/mri/SNS_MRI_DNUMR_S000${sub}_01/*_ps_bvalue1_diffori33.nii


## 4.	Get distortion corrections:
--> run on sciencecloud, takes long!

[topup --imain=b0_all --datain=acqparams.txt --config=b02b0.cnf --subsamp=1 --out=topup_results --iout=corrected_b0]

base_folder=/mnt_AdaBD_largefiles/Data/SMILE_DATA/DNumRisk/ds-dnumrisk/derivatives/dwi_preproc

topup --imain=${base_folder}/sub-${sub}/sub-${sub}_dir-all-b0_dwi.nii.gz \
--datain=${base_folder}/acqparams.txt --config=b02b0.cnf --subsamp=1 \
--out=${base_folder}/sub-${sub}/topup_res \
--iout=${base_folder}/sub-${sub}/sub-${sub}_corrected_b0

--> tried to apply it to all DWI-ims (directly) sub-01_dir-all_dwi.nii.gz), but does not work...


#### locally --> tried to apply it to all DWI-ims (directly) sub-01_dir-all_dwi.nii.gz), but does not work...
[adding `-b0`to imain makes it work again]

topup --imain=/Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-01/sub-01_dir-all_dwi.nii.gz \
--datain=/Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/acqparams.txt \
--config=b02b0.cnf --subsamp=1 \
--out=/Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-01/topup_res \
--iout=/Volumes/mrenkeED/data/ds-dnumrisk/derivatives/dwi_preproc/sub-01/corrected_b0

## 5.	Apply topup results: 

applytopup --imain=dwi_all --topup=topup_results --datain=acqparams.txt --inindex=1 --method=jac --out=dwi_topup_corrected

applytopup --imain=${base_folder}/sub-${sub}/sub-${sub}_dir-all_dwi.nii.gz \
--topup=${base_folder}/sub-${sub}/topup_res \
--datain=${base_folder}/acqparams.txt --inindex=1 --method=jac \
--out=${base_folder}/sub-${sub}/sub-${sub}_dir-all_topup-corr_dwi



## 6.	Get brain mask 

from mean dti image (the version where I generate the brain mask from corrected_b0 seems to misalign) :
    a.	fslmaths dwi_topup_corrected.nii.gz -Tmean dwi_topup_corrected_mean
    b.	bet dwi_topup_corrected_mean.nii.gz mask_dwi_topup_corrected_mean -m -f 0.2

--> zoltan said I should check, each, play around with the params
--> -f 0.4 seemed best for sub-01 so far...

1. fslmaths ${base_folder}/sub-${sub}/sub-${sub}_dir-all_topup-corr_dwi -Tmean ${base_folder}/sub-${sub}/sub-${sub}_dir-all_topup-corr_mean_dwi
2. bet ${base_folder}/sub-${sub}/sub-${sub}_dir-all_topup-corr_mean_dwi ${base_folder}/sub-${sub}/mask-f04_sub-${sub}_dir-all_topup-corr_mean_dwi  -m -f 0.4

# 7.	Fit diffusion tensor model at each voxel:

dtifit --data=dwi_topup_corrected --mask=mask_dwi_topup_corrected_mean_mask --bvecs=dwi_all.bvecs --bvals=dwi_all.bvals --out=dtifit_meanmask_results

dtifit --data=${base_folder}/sub-${sub}/sub-${sub}_dir-all_topup-corr_dwi \
--mask=${base_folder}/sub-${sub}/mask-f04_sub-${sub}_dir-all_topup-corr_mean_dwi_mask \
--bvecs=${base_folder}/sub-${sub}/sub-${sub}_dir-all_dwi.bvecs \
--bvals=${base_folder}/sub-${sub}/sub-${sub}_dir-all_dwi.bvals \
--out=${base_folder}/sub-${sub}/sub-${sub}_dtifit_res


# Alignment & group level Statistics

## 1. align each subject's FA map to a standard template (like MNI152)

1. coregister the DWI to the T1 - `flirt` or `epi_reg` 
2. Use `fsl_anat` or `fnirt` on the structural (T1-weighted) images, then apply the warp to the FA.

fsl_anat -i sub-01_ses-1_T1w.nii [performs also fnirt?!]

flirt -in fa.nii.gz \
      -ref T1.nii.gz \
      -omat fa2t1.mat \
      -out fa_in_T1.nii.gz \
      -cost bbr \
      -wmseg <white_matter_mask_from_fsl_anat> \
      -dof 6 \
      -init identity.mat \
      -schedule ${FSLDIR}/etc/flirtsch/bbr.sch

T1_fast_pve_2.nii.gz


applywarp -i fa_in_T1.nii.gz \
          -r <MNI152_T1_1mm.nii.gz> \
          -w T1_to_MNI_warp.nii.gz \
          -o fa_in_MNI.nii.gz


### what did not work so far...
epi_reg --epi=sub-01_dtifit_res_FA.nii.gz --t1=sub-01_ses-1_T1w.nii --t1brain=sub-01_ses-1_T1w_masked-brain.nii.gz --out=epi_reg_res --wmseg=T1_fast_pve_1.nii.gz

 ❯ flirt -in sub-01_dtifit_res_FA.nii.gz -ref T1_biascorr.nii.gz -applyxfm -init fa2t1_epi_reg.mat -out fa_in_T1.nii.gz

flirt -in sub-01_dtifit_res_FA \
    -ref anatT1/sub-01_ses-1_T1w.nii \
    -omat fa2t1.mat \
    -out fa_in_T1.nii.gz \
    -cost bbr \
    -wmseg anatT1/sub-01_ses-1_T1w.anat/T1_fast_pve_2.nii.gz




    -schedule ${FSLDIR}/etc/flirtsch/bbr.sch
    -dof 6 \
    -init identity.mat \


## OLD

In [ ]:
# Tranpose files | not needed !

fn_in = '/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-01/bvals_all.txt'
fn_out = '/Volumes/mrenkeED/data/ds-dnumrisk/sourcedata/tryout_DTI/sub-01/bvals_all_T.txt'
# Read the file
with open(fn_in, "r") as file:
    lines = [line.strip().split() for line in file]

# Transpose the data
transposed = list(zip(*lines))

# Write the transposed data to a new file
with open(fn_out, "w") as file:
    for row in transposed:
        file.write(" ".join(row) + "\n")
